In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# wd = '/mnt/openfact/users/msawinski/factue-task2'
wd = '/Users/marcinsawinski/Documents/GitHub/factue-task2'
import sys, os
os.chdir(wd)

In [3]:
import pandas as pd
from pathlib import Path
import os
from factue.utils.viz import disp
import numpy as np

# extract


In [4]:

root = Path("data/llm_output/normalization/extract")
# Example path template
files = root.rglob("**/*.parquet")

# Read all files and add file path as a column
df_list = []
for f in files:
    df_part = pd.read_parquet(f)
    df_part['source_file'] = f  # add the file path
    df_list.append(df_part)

# Combine into one DataFrame
df = pd.concat(df_list, ignore_index=True)
df['split'] = df.source_file.astype(str).str.split('/',expand=True)[7]
# df['init_part_no'] = df.source_file.astype(str).str.split('/',expand=True)[9].str.split('_',expand=True)[1].str.split('.',expand=True)[0].astype(int)
df['lang'] = df.source_file.astype(str).str.split('/',expand=True)[8]
df['split'].value_counts()

split
dev    10
Name: count, dtype: int64

In [5]:
df

,text,reference,split,lang,init_part_no,output_id,prompt_name,prompt_version,job,step,...,model_name,model_mode,plain_content,status,is_valid,raw,think_content,final,error,source_file
0,Sikh with no mustache in Delhi farmers protest. According to astronomy such an event happens once in a century,'Sardar with no mustache' involved in farmers protest,dev,dev-eng,43,LLAMA_31_8B/v001/default,default,v001,normalization,extract,...,llama3.1:8b,chat,"[A Sikh with no mustache is present at the Delhi farmers protest, which astronomers say occurs only once per century., A Sikh with no mustache is present at the Delhi farmers protest, which astron...","[no validation, no validation]","[True, True]","[A Sikh with no mustache is present at the Delhi farmers protest, which astronomers say occurs only once per century., A Sikh with no mustache is present at the Delhi farmers protest, which astron...","[None, None]","A Sikh with no mustache is present at the Delhi farmers protest, which astronomers say occurs only once per century.",None,data/llm_output/normalization/extract/LLAMA_31_8B/v001/default/dev/dev-eng/dev-eng_part_0000.parquet
1,"The government advice is not to challenge people to wear a face covering. This is for GOOD REASON. If you do so, you and your employees may be PERSONALLY LIABLE for AN OFFENCE liable on a summary ...",The government advice is not to challenge someone who is not wearing a face covering.,dev,dev-eng,31,LLAMA_31_8B/v001/default,default,v001,normalization,extract,...,llama3.1:8b,chat,"[Challenging people to wear a face covering may result in personal liability for an offence punishable by a fine of up to £5,000., Challenging people to wear a face covering may result in personal...","[no validation, no validation]","[True, True]","[Challenging people to wear a face covering may result in personal liability for an offence punishable by a fine of up to £5,000., Challenging people to wear a face covering may result in personal...","[None, None]","Challenging people to wear a face covering may result in personal liability for an offence punishable by a fine of up to £5,000.",None,data/llm_output/normalization/extract/LLAMA_31_8B/v001/default/dev/dev-eng/dev-eng_part_0000.parquet
2,Will our BJP\ngovernment return in 2024!!!,Will our BJP government return in 2024,dev,dev-eng,34,LLAMA_31_8B/v001/default,default,v001,normalization,extract,...,llama3.1:8b,chat,"[Our BJP government will return in 2024!!!, Our BJP government will return in 2024!!!]","[no validation, no validation]","[True, True]","[Our BJP government will return in 2024!!!, Our BJP government will return in 2024!!!]","[None, None]",Our BJP government will return in 2024!!!,None,data/llm_output/normalization/extract/LLAMA_31_8B/v001/default/dev/dev-eng/dev-eng_part_0000.parquet
3,Ok truth out #TriptiDesai converted to christianity three years back..in 2013..closely connected to conversion racket...she is a CHRISTIAN in hindu garb...beware keralites..come to destroy #Sabari...,Trupti Desai Converted To Christianity,dev,dev-eng,26,LLAMA_31_8B/v001/default,default,v001,normalization,extract,...,llama3.1:8b,chat,"[Truth out #TriptiDesai converted to Christianity three years back, closely connected to conversion racket. She is a Christian in Hindu garb..., Truth out #TriptiDesai converted to Christianity th...","[no validation, no validation]","[True, True]","[Truth out #TriptiDesai converted to Christianity three years back, closely connected to conversion racket. She is a Christian in Hindu garb..., Truth out #TriptiDesai converted to Christianity th...","[None, None]","Truth out #TriptiDesai converted to Christianity three years back, closely connected to conversion racket. She is a Christian in Hindu garb...",None,data/llm_output/normalization/extract/LLAMA_31_8B/v001/default/dev/dev-eng/dev-eng_part_0000.parquet
4,"MUGABE'S CASEKET\n17billion worth of Robert Mugabe's casket\nhas been computerized in such a way that after his interment, his family can watch 

In [6]:
sample  = df[['prompt_version','model_name','text','reference', 'final','plain_content', 'think_content']].sort_values(['text',"model_name", 'prompt_version'])
sample

,prompt_version,model_name,text,reference,final,plain_content,think_content
6,v001,llama3.1:8b,"Just to let you know that the first thing that’ll strike you when you look up this act on the government website is that it says changes to legislation, and this has been updated on the 27th April...",The Control of Diseases Act in the UK changed on 27 April.,"Changes to legislation have come into force on April 27th, 2020.","[Changes to legislation have come into force on April 27th, 2020., Changes to legislation have come into force on April 27th, 2020.]","[None, None]"
4,v001,llama3.1:8b,"MUGABE'S CASEKET\n17billion worth of Robert Mugabe's casket\nhas been computerized in such a way that after his interment, his family can watch his remains through their phone devices...\n\nAccord...",Mugabe has computerised coffin,Robert Mugabe's casket has been computerized so his family can watch his remains through their phone devices after interment.,"[Robert Mugabe's casket has been computerized so his family can watch his remains through their phone devices after interment., Robert Mugabe's casket has been computerized so his family can watch...","[None, None]"
3,v001,llama3.1:8b,Ok truth out #TriptiDesai converted to christianity three years back..in 2013..closely connected to conversion racket...she is a CHRISTIAN in hindu garb...beware keralites..come to destroy #Sabari...,Trupti Desai Converted To Christianity,"Truth out #TriptiDesai converted to Christianity three years back, closely connected to conversion racket. She is a Christian in Hindu garb...","[Truth out #TriptiDesai converted to Christianity three years back, closely connected to conversion racket. She is a Christian in Hindu garb..., Truth out #TriptiDesai converted to Christianity th...","[None, None]"
0,v001,llama3.1:8b,Sikh with no mustache in Delhi farmers protest. According to astronomy such an event happens once in a century,'Sardar with no mustache' involved in farmers protest,"A Sikh with no mustache is present at the Delhi farmers protest, which astronomers say occurs only once per century.","[A Sikh with no mustache is present at the Delhi farmers protest, which astronomers say occurs only once per century., A Sikh with no mustache is present at the Delhi farmers protest, which astron...","[None, None]"
5,v001,llama3.1:8b,"Steve Kirsch, the Executive Director of the COVID-19 Early Treatment Fund was up next to testify before the FDA. Kirsch argued the vaccines kill more people than they save.Steve Kirsch also point...",Covid-19 vaccines kill more people then they save,Government data demonstrates that COVID shots may kill more than they save.,"[Government data demonstrates that COVID shots may kill more than they save., Government data demonstrates that COVID shots may kill more than they save.]","[None, None]"
7,v001,llama3.1:8b,"The London Mayor says we should create harmony by learning Arabic. He wants us to learn more about Muslims, their religion, and accept their culture.",Mayor of London Sadiq Khan said people should learn Arabic.,London Mayor suggests learning Arabic to promote understanding of Muslim culture and acceptance.,"[London Mayor suggests learning Arabic to promote understanding of Muslim culture and acceptance., London Mayor suggests learning Arabic to promote understanding of Muslim culture and acceptance.]","[None, None]"
1,v001,llama3.1:8b,"The government advice is not to challenge people to wear a face covering. This is for GOOD REASON. If you do so, you and your employees may be PERSONALLY LIABLE for AN OFFENCE liable on a summary ...",The government advice is not to challenge someone who is not wearing a face covering.,"Challenging people to wear a face covering may result in personal liability for an offence punishable by a fine of up to £5,000.","[Challenging people to wear a face covering may result in personal liability for an offence punishable by a fine of up to £5,000., Challenging people to wear a face covering may result in perso

In [7]:
disp(sample)

,prompt_version,model_name,text,reference,final,plain_content,think_content
6,v001,llama3.1:8b,"Just to let you know that the first thing that’ll strike you when you look up this act on the government website is that it says changes to legislation, and this has been updated on the 27th April 2020. So just a day or so ago. And it’s come into force on that day. So this is applicable from now.",The Control of Diseases Act in the UK changed on 27 April.,"Changes to legislation have come into force on April 27th, 2020.","['Changes to legislation have come into force on April 27th, 2020.' 'Changes to legislation have come into force on April 27th, 2020.']",[None None]
4,v001,llama3.1:8b,"MUGABE'S CASEKET 17billion worth of Robert Mugabe's casket has been computerized in such a way that after his interment, his family can watch his remains through their phone devices... According to the London Casket makers that supplied the casket,the facilities in it will protect his remains from decomposing for the next ten years...this however implies that,apart from relating with his family, any member of his family that wishes to watch his remains in the tomb will have to access same via his or her phone devices. The white is still at the top using our brains MUGABE'S CASEKET 17billion worth of Robert Mugabe's casket has been computerized in such a way that after his interment, his family can watch his remains through their phone devices... According to the London Casket makers that supplied the casket,the facilities in it will protect his remains from decomposing for the next ten years...this however implies that,apart from relating with his family, any member of his family that wishes to watch his remains in the tomb will have to access same via his or her phone devices. The white is still at the top using our brains MUGABE'S CASEKET 17billion worth of Robert Mugabe's casket has been computerized in such a way that after his interment, his family can watch his remains through their phone devices... According to the London Casket makers that supplied the casket,the facilities in it will protect his remains from decomposing for the next ten years...this however implies that,apart from relating with his family, any member of his family that wishes to watch his remains in the tomb will have to access same via his or her phone devices. The white is still at the top using our brains None",Mugabe has computerised coffin,Robert Mugabe's casket has been computerized so his family can watch his remains through their phone devices after interment.,"[""Robert Mugabe's casket has been computerized so his family can watch his remains through their phone devices after interment."" ""Robert Mugabe's casket has been computerized so his family can watch his remains through their phone devices after interment.""]",[None None]
3,v001,llama3.1:8b,Ok truth out #TriptiDesai converted to christianity three years back..in 2013..closely connected to conversion racket...she is a CHRISTIAN in hindu garb...beware keralites..come to destroy #Sabarimala #JanamTV report exposes this..,Trupti Desai Converted To Christianity,"Truth out #TriptiDesai converted to Christianity three years back, closely connected to conversion racket. She is a Christian in Hindu garb...","['Truth out #TriptiDesai converted to Christianity three years back, closely connected to conversion racket. She is a Christian in Hindu garb...' 'Truth out #TriptiDesai converted to Christianity three years back, closely connected to conversion racket. She is a Christian in Hindu garb...']",[None None]
0,v001,llama3.1:8b,Sikh with no mustache in Delhi farmers protest. According to astronomy such an event happens once in a century,'Sardar with no mustache' involved in farmers protest,"A Sikh with no mustache is present at the Delhi farmers protest, which astronomers say occurs only once per century.","['A Sikh with no mustache is present at the Delhi farmers protest, which astronomers say occurs only once per ce